In [1]:
# TESTE ESTATÍSTICO DE HIPÓTESE

"""
    De forma geral, as estatísticas devem ser vistas como observações
        de variáveis aleatórias a partir de distribuições conhecidas, 
        o que permite que façamos declarações sobre as premissas mais
        prováveis de serem corretas.

    Em uma hipótese clássica, temos a hipótese nula H0 que representa 
        uma posição padrão, e alguma hipótese H1 com a qual gostariamos
        de compará-la.
    """

# EXEMPLO: LANÇAR UMA MOEDA

"""
    Imagine qe temos uma moeda e queremos testar para confirmar
        se ela é honesta.
    Temos a premissa de que a moeda possui a probabilidade p de cair
        'cara', então nossa hipótese nula é que a moeda seja honesta.
        Ou seja, p = 0.5

    Em especial, nosso teste envolverá o lançamento da moeda em número n 
        de vezes e contando o número de caras X. Cada lançamento da moeda
        é uma Tentativa de Bernoulli, o que significa que X é uma variável
        aleatória Binomial(n, p), que podemos aproximar usando a 
        distribuição normal
    """
from typing import Tuple
import math

def normal_approximation_to_binomial(n:int , 
                                     p: float) -> Tuple[float, float]:
    """Encontra mu e sigma correspondendo ao Binomial(n, p)"""
    mu = p*n
    sigma = math.sqrt(mu * (1-p))
    return mu, sigma                # retorna os parâmetros para uma distribuição normal de (n, p)

from probability import normal_cdf

# o cdf normal é a probabilidade que a variável esteja abaixo de um limite
normal_probability_below = normal_cdf

# está acima do limite se não estiver abaixo
def normal_probability_above(lo: float, 
                             mu: float = 0, 
                             sigma: float = 1) -> float:
    return 1-normal_cdf(lo, mu, sigma)

# está entre se for menos do que hi, mas não menor do que lo
def normal_probability_betwenn(lo: float, 
                               hi: float, 
                               mu: float = 0, 
                               sigma: float = 1) -> float:
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# está fora se não estiver entre
def normal_probability_outside(lo: float, 
                               hi: float, 
                               mu: float = 0, 
                               sigma: float = 1) -> float:
    return 1 - normal_probability_betwenn(lo, hi, mu, sigma)

"""
    Agora, o contrário - encontrar a região sem intervalo (simétrico)
        em torno da média que contribui para o nível de probabilidade.
    Por exemplo, para encontrar o intervalo centrado na média que contém 60%
        da probabilidade, determina-se limiares inferiores e superiores, de 
        modo a excluir 20% de cada um...
    """

from probability import inverse_normal_cdf

def normal_upper_bound(probability: float, 
                       mu: float = 0, 
                       sigma: float = 1) -> float:
    """retorna z para que P(Z <= z) = probability"""
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability: float, 
                       mu: float = 0, 
                       sigma: float = 1) -> float:
    """retorna z para que P(Z >= z) = probability"""
    return inverse_normal_cdf(1-probability, mu, sigma)

def normal_two_sided_bounds(probability: float, 
                            mu: float = 0, 
                            sigma: float = 1) -> Tuple[float, float]:
    """retorna os limites simétricos (sobre a média) que contêm a 
        probabilidade específica"""
    tail_probability = (1-probability)/2

    # limite superior deveria ter tail_probability acima
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    # limite inferior deveria ter tail_probability abaixo
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)

    return lower_bound, upper_bound

"""
    Em especial, se escolhemos lançar uma moeda n = 1000 vezes.
        Se nossa hipótese de honestidade for verdadeira, X deveria ser
        distribuído normalmente com média 500 e desvio padrão de 15.8
    """

mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)

"""
    Precisamos tomar uma decisão sobre significância:
        De quanto é a vontade que temos de fazer um erro tipo 1 ("falso
        positivo"), em que rejeitamos H0 mesmo se for verdadeiro. 
        Considera-se: 1% a 5%
    """

normal_two_sided_bounds(0.95, mu_0, sigma_0)

"""    
    Também é interessante considerar a probabilidade de não cometer um
        erro do tipo 2, no qual falhamos em rejeitar H0 mesmo ele sendo
        falso.
    """

# 95% dos limites baseados na premissa P é 0,5
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

# mu e sigma reais baseados em P = 0,55
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

# Um erro tipo 2 significa que falhamos ao rejeitar a hipótese nula
# o que acontecerá quando X ainda estiver em nosso intervalo original
type_2_probability = normal_probability_betwenn(lo, hi, mu_1, sigma_1)
power = 1-type_2_probability

"""
    Imaginemos que a nossa hipótese nula fosse que a moeda não seria
        inclinada a 'cara', ou que P < 0.5.
    Nesse caso, queríamos um teste unilateral que rejeitasse a hipótese
        nula quando X fosse muito maior que 50, mas não quando X fosse menor.
    Portanto, um teste de significância de 5% envolveria usar 
        normal_probability_below para encontrar o corte abaixo dos 95% em
        que a probabilidade ficaria:
    """

hi = normal_upper_bound(0.95, mu_0, sigma_0)
type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability

<Figure size 640x480 with 0 Axes>

In [3]:
# P-VALUES

import random

"""
    Outra forma de fazer a mesma coisa
    No lugar de escolher limites com base em alguma probabilidade de corte
    Computamos a probabilidade que podemos ver um valor ao menos
    tão extremo quanto ao que realmente observamos.
    """

def two_sided_p_value(x: float, 
                      mu: float = 0, 
                      sigma: float = 1) -> float:
    if x >= mu:
        # se x for maior do que a média, a coroa será o que for maior do que x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # se x for menor que a média, a coroa será o que menor do que x
        return 2 * normal_probability_below(x, mu, sigma)

two_sided_p_value(529.5, mu_0, sigma_0)

from inference import normal_probability_above, normal_probability_below

# http://en.wikipedia.org/wiki/Continuity_correction

extreme_value_count = 0 

for i in range(1000):
    num_heads = sum(
        1 
        if random.random() < 0.5 
        else 0 
        for _ in range(1000))

    if num_heads >= 530 or num_heads <= 470:
        extreme_value_count += 1

assert 59 < extreme_value_count < 65, f"{extreme_value_count}"

# Desde que p-value seja maior do que a significância de 5%, não 
# rejeitamos a hipótese nula.

two_sided_p_value(531.5, mu_0, sigma_0)

upper_p_value = normal_probability_above
lower_p_value = normal_probability_below

upper_p_value(524.5, mu_0, sigma_0)

upper_p_value(526.5, mu_0, sigma_0)

0.04686839508859242

In [14]:
# INTERVALOS DE CONFIANÇA

import math

from inference import normal_two_sided_bounds

p_hat = 525/1000
mu = p_hat
sigma = math.sqrt(p_hat*(1-p_hat)/1000)

normal_two_sided_bounds(0.95, mu, sigma)

p_hat = 540/1000
mu = p_hat
sigma = math.sqrt(p_hat*(1-p_hat)/1000)

normal_two_sided_bounds(0.95, mu, sigma)

(0.5091095927295919, 0.5708904072704082)

In [4]:
# P-HACKING

import random
from typing import List

def run_experiment() -> List[bool]:
    """lança uma moeda 1000 vezes, True = 'cara', False = 'coroa'"""
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment: List[bool]) -> bool:
    """usando 5% dos níveis de significância"""
    num_heads = len([flip 
                     for flip in experiment 
                     if flip])
    return num_heads < 469 or num_heads > 531

random.seed(0)

experiments = [run_experiment() 
               for _ in range(1000)]

num_rejections = len([experiment 
                      for experiment in experiments 
                      if reject_fairness(experiment)])

assert num_rejections == 46

# http://bit.ly/1L2QtCr

In [3]:
# EXEMPLO: EXECUTAND UM TESTE A/B

import math
from inference import two_sided_p_value

def estimated_parameters (N, n):
    p = n/N
    sigma = math.sqrt(p*(1-p)/N)
    return p, sigma

def a_b_test_statistic(N_A: int, 
                       n_A: int, 
                       N_B: int, 
                       n_B: int) -> float:
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A)/math.sqrt(sigma_A**2 + sigma_B**2)

z = a_b_test_statistic(1000, 200, 1000, 180)

two_sided_p_value(z)

z = a_b_test_statistic(1000, 200, 1000, 150)

two_sided_p_value(z)

0.003189699706216853

<Figure size 640x480 with 0 Axes>

In [ ]:
# INFERÊNCIA BAYNESIANA

"""
    Uma alternativa à inferência é tratar os parâmetros desconhecidos como variáveis aleatórias.
    
    Partindo de uma distribuição anterior para definir os parâmetros e utilizando os dados
        observados e o teorema de Bayes para atualizar a distribuição posterior desses parâmetros.
    
    Ou seja, ao invés de emitir declarações de probabilidade sobre os teste, avalia-se
    a probabilidade dos parâmetros.
    """

import math

def B(alpha: float, beta: float) -> float:
    """Uma constante normalizadora para a qual a probabilidade total é 1"""
    return (math.gamma(alpha) * math.gamma(beta)) / math.gamma(alpha + beta)

def beta_pdf (x: float, alpha: float, beta: float) -> float:
    if x <= 0 or x >= 1:
        return 0
    return x**(alpha - 1) * (1-x) ** (beta-1) / B(alpha, beta)

# Em geral, essa distribuição centraliza seu peso em: alpha / (alpha + beta)

"""
    Imagine que partimos de uma distribuição anterior em p.
    Suponha que lançamos a moeda várias vezes e observamos h 'caras' e t 'coroas'
    Segundo Bayes:
        A distribuição posterior e p é uma distribuição Beta, mas com os parâmetros alpha + h e beta + t
    """

# http://www.johndcook.com/blog/conjugate_prior_diagram/

# https://www.coursera.org/course/statistics